# Presets

We you call `fit`, AutoGluon will explore a set of models to save your time on manual hyperparameter tuning. The more models to explore, the better accuracy you often get. But it also leads to more computational cost. There are several ways to balance the model accuracy and computational cost. The easiest one is through the `presets` argument in the `fit` method.

A preset setting specifies a particular set of models and how they are combined for prediction. AutoGluon provides 4 presets: `medium_quality`, `good_quality`, `high_quality`, and `best_quality`. The differences are listed in the following table. 

| Preset | Mode Quality | Fit Time | Predict Time | Disk Usage | Use Cases | 
|:---|:---|:---|:---|:---|:---|
| `best_quality` | Best | 16x  | 32x | 16x | When accuracy is what matters |
| `high_quality` | High | 16x  | 4x | 2x | When you need a very powerful solution with fast (batch) inference |
| `good_quality` | Good | 16x | 2x | 1x | When a powerful, highly portable solution with very fast inference is required: Billion-scale batch inference, sub-100ms online-inference, edge-devices |
| `medium_quality` | Medium | 1x | 1x | 1x | Initial prototyping, establishing a performance baseline |

We recommend you to start with `medium_quality`, which is the default setting, to get a sense of the problem and identify any data related issues. It's the fastest option. You can further accelerate it by subsampling your data or specifying a proper `time_limit` argument for the `fit` method. 

Once you are comfortable, next try `best_quality`. Make sure to specify at least 16x the `time_limit` value as used in `medium_quality`. Once finished, you should have a very powerful solution that is often stronger than `medium_quality`, especially for complex data. 

Once you evaluate both `best_quality` and `medium_quality`, check if either satisfies your needs. If neither do, consider trying `high_quality` and/or `good_quality`. 

Now let's train a model with the `high_quality` preset and evaluate its performance.

In [1]:
#@title Install autogluon
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 203 kB 3.8 MB/s 
     |████████████████████████████████| 48 kB 4.2 MB/s 
     |████████████████████████████████| 61 kB 246 kB/s 
     |████████████████████████████████| 59 kB 1.3 MB/s 
     |████████████████████████████████| 141 kB 8.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 272 kB 9.3 MB/s 
     |████████████████████████████████| 802 kB 9.1 MB/s 
     |████████████████████████████████| 1.0 MB 21.7 MB/s 
     |████████████████████████████████| 132 kB 32.2 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.6 MB 27.0 MB/s 
     |████████████████████████████████| 54.5 MB 1.4 MB/s 
     |████████████████████████████████| 281 kB 33.9 MB/s 
     |████████████████████████████████| 13.5 MB 52.9 MB/s 
     |████████████████████████████████| 1.

In [5]:
from autogluon.tabular import TabularDataset, TabularPredictor

url = 'https://autogluon.s3.amazonaws.com/datasets/Inc/'
train_data = TabularDataset(url+'train.csv')
test_data = TabularDataset(url+'test.csv')

predictor = TabularPredictor(label='class').fit(
    train_data, presets='high_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20220707_180104/"
Presets specified: ['high_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220707_180104/"
AutoGluon Version:  0.5.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    39073
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' <=50K', ' >50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either re

In [4]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.8774695465247211,
 'balanced_accuracy': 0.7975275683791567,
 'f1': 0.7142516113630938,
 'mcc': 0.643318065933343,
 'precision': 0.7995724211651524,
 'recall': 0.6453839516824849,
 'roc_auc': 0.9327997272719587}

You can see the accuracy is slighted increased compared to the default `medium_quality` preset in :doc:`/get_started/tabular_quick_start`. But note that we are using a very simple dataset, this small difference is expected.


Finally, if none of the presets satisfy your requirements, can you manually specify the set of models to fit with their hyperparameters. Refer to TODO.